In this notebook we explore sets by manual implementation, with a focus on time complexity of the algorithms.

In [2]:
import time

RUNTIME_OVERHEAD = 0.0

def runtime(f, name=None, verbose=True, prewarm=None, iterations=1, **kwargs):
    prewarm_results = [prewarm(**kwargs) for _ in range(iterations)] if prewarm else [dict() for _ in range(iterations)]
    start = time.time()
    results = [None] * iterations
    for i in range(iterations):
        if prewarm is not None:
            kwargs.update(prewarm_results[i])
        results[i] = f(**kwargs)
    end = time.time()
    runtime = end - start - RUNTIME_OVERHEAD
    if verbose:
        print(f"Runtime {'of ' + name if name else ''}: {runtime} seconds")
    if any(result is not None for result in results):
        return results if len(results) > 1 else results[0], runtime / iterations
    return None, runtime / iterations

def waitloop(seconds):
    start = time.time()
    while time.time() - start < seconds:
        pass
    
time_wait = 0.15
_, rt = runtime(waitloop, verbose=False, iterations=5, seconds=time_wait)
# RUNTIME_OVERHEAD = rt + RUNTIME_OVERHEAD - time_wait
print(f"Average overhead time over 5 runs: {rt + RUNTIME_OVERHEAD - time_wait} seconds")

Average overhead time over 5 runs: 0.0001652240753173828 seconds


In [ ]:
class Element:
    def __init__(self, key, data):
        self.key = key
        self.data = data
        self.left = None
        self.right = None
        self.height = 1

class SetADT:
    def __init__(self, enable_balancing=True):
        self.ENABLE_BALANCING = enable_balancing
        self.root = None

    def build(self, X):
        for k, d in X:
            self.insert(k, d)
        return self
            
    def height(self, n):
        return n.height if n else 0

    def insert(self, key, data):
        self.root = self._insert(self.root, key, data)

    def _insert(self, node, key, data):
        if not node:
            return Element(key, data)
        if key < node.key:
            node.left = self._insert(node.left, key, data)
        elif key > node.key:
            node.right = self._insert(node.right, key, data)
        else:
            return node  # already exists

        node.height = 1 + max(self.height(node.left), self.height(node.right))
        return self._rebalance(node, key)

    def delete(self, key):
        self.root = self._delete(self.root, key)

    def _delete(self, node, key):
        if not node:
            return node

        if key < node.key:
            node.left = self._delete(node.left, key)
        elif key > node.key:
            node.right = self._delete(node.right, key)
        else:
            if not node.left:
                return node.right
            if not node.right:
                return node.left

            # Find smallest element larger than node
            temp = node.right
            while temp.left:
                temp = temp.left
            node.key, node.data = temp.key, temp.data
            node.right = self._delete(node.right, temp.key)

        node.height = 1 + max(self.height(node.left), self.height(node.right))
        return self._rebalance(node, key)

    def find(self, key):
        node = self.root
        while node:
            if key == node.key:
                return node.key, node.data, True
            node = node.left if key < node.key else node.right
        return None, None, False

    def find_min(self):
        if not self.root:
            return None
        node = self.root
        while node.left:
            node = node.left
        return node.key, node.data

    def find_max(self):
        if not self.root:
            return None
        node = self.root
        while node.right:
            node = node.right
        return node.key, node.data

    def find_next(self, key):
        curr = self.root
        succ = None
        while curr:
            if key < curr.key:
                succ = curr
                curr = curr.left
            else:
                curr = curr.right
        return (succ.key, succ.data) if succ else None

    def find_prev(self, key):
        curr = self.root
        pred = None
        while curr:
            if key > curr.key:
                pred = curr
                curr = curr.right
            else:
                curr = curr.left
        return (pred.key, pred.data) if pred else None
    
    def _rebalance(self, node, key):
        if self.ENABLE_BALANCING is False:
            return node
        balance = self.height(node.left) - self.height(node.right)

        # LL
        if balance > 1 and key < node.left.key:
            return self._rotate_right(node)

        # RR
        if balance < -1 and key > node.right.key:
            return self._rotate_left(node)

        # LR
        if balance > 1 and key > node.left.key:
            node.left = self._rotate_left(node.left)
            return self._rotate_right(node)

        # RL
        if balance < -1 and key < node.right.key:
            node.right = self._rotate_right(node.right)
            return self._rotate_left(node)

        return node

    def _rotate_right(self, y):
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        y.height = 1 + max(self.height(y.left), self.height(y.right))
        x.height = 1 + max(self.height(x.left), self.height(x.right))
        return x

    def _rotate_left(self, x):
        y = x.right
        T2 = y.left
        y.left = x
        x.right = T2
        x.height = 1 + max(self.height(x.left), self.height(x.right))
        y.height = 1 + max(self.height(y.left), self.height(y.right))
        return y

Testing the set:

In [4]:
S = SetADT()
S.build([(5,"a"), (3,"b"), (8,"c"), (1,"d")])

print(S.find(3))       # (3, 'b', True)
print(S.find_min())    # (1, 'd')
print(S.find_max())    # (8, 'c')
print(S.find_next(3))  # (5, 'a')
print(S.find_prev(5))  # (3, 'b')

S.delete(3)
print(S.find(3))       # (None, None, False)


(3, 'b', True)
(1, 'd')
(8, 'c')
(5, 'a')
(3, 'b')
(None, None, False)


Time complexity of methods:
- build: O(n.O(insert)) = O(n.lgn) 
- find: O(lgn) due to balanced tree (AVL) with binary decisions, ie lg_2(n). Otherwise O(h), with h generally > lg_2(n).
- insert: O(lgn) since we iterate to bottom of tree of height lgn.
- delete: O(lgn) same reason, we need to go to bottom of tree to find replacement for deleted node.
- find_min: O(lgn) go all the way to the left.
- find_max: O(lgn) go all the way to the right.
- find_prev: O(lgn) because the prev node can never be more than the tree height away.
- find_next: O(lgn) because the next node can never be more than the tree height away.

In [29]:
# Benchmark
def get_random_pairs(n, key_range, sort=False):
    import random
    pairs = set()
    while len(pairs) < n:
        key = random.randint(1, key_range)
        data = random.random()
        pairs.add((key, data))
    if sort:
        return sorted(pairs)
    return list(pairs)

def build_random_set(n=1000, key_range=5000, enable_balancing=True, sort=False):
    return SetADT(enable_balancing=enable_balancing).build(get_random_pairs(n, key_range, sort=sort))

# Testing unbalanced vs balanced build times
runtime(
    build_random_set,
    name="Unbalanced build",
    iterations=10,
    enable_balancing=False,
    )
runtime(
    build_random_set,
    name="Balanced build",
    iterations=10,
    enable_balancing=True,
    )

# Testing unbalanced vs balanced find_min times
runtime(
    lambda set: set.find_min(),
    prewarm=lambda: {'set': build_random_set(enable_balancing=False)},
    name="Unbalanced find_min",
    iterations=100
    )

runtime(
    lambda set: set.find_min(),
    prewarm=lambda: {'set': build_random_set(enable_balancing=True)},
    name="Balanced find_min",
    iterations=100
    )

# Testing unbalanced vs balanced insert times
runtime(
    lambda set, pair: set.insert(*pair),
    prewarm=lambda: {
        'set': build_random_set(enable_balancing=False),
        'pair': get_random_pairs(1, 5000)[0]
    },
    name="Unbalanced insert",
    iterations=100
    )
_=runtime(
    lambda set, pair: set.insert(*pair),
    prewarm=lambda: {
        'set': build_random_set(enable_balancing=True),
        'pair': get_random_pairs(1, 5000)[0]
    },
    name="Balanced insert",
    iterations=100
    )

# Testing unbalanced vs balanced insert times
runtime(
    lambda set, pair: set.insert(*pair),
    prewarm=lambda: {
        'set': build_random_set(enable_balancing=False, sort=True),
        'pair': get_random_pairs(1, 5000)[0]
    },
    name="Unbalanced insert (sorted)",
    iterations=10
    )
_=runtime(
    lambda set, pair: set.insert(*pair),
    prewarm=lambda: {
        'set': build_random_set(enable_balancing=True, sort=True),
        'pair': get_random_pairs(1, 5000)[0]
    },
    name="Balanced insert (sorted)",
    iterations=10
    )

Runtime of Unbalanced build: 0.28907108306884766 seconds
Runtime of Balanced build: 0.26571035385131836 seconds
Runtime of Unbalanced find_min: 0.0 seconds
Runtime of Balanced find_min: 0.0 seconds
Runtime of Unbalanced insert: 0.0030794143676757812 seconds
Runtime of Balanced insert: 0.002992391586303711 seconds
Runtime of Unbalanced insert (sorted): 0.011019229888916016 seconds
Runtime of Balanced insert (sorted): 0.0 seconds


We see that the time to balance a tree with 1000 nodes takes roughly 0.04 seconds, and does actually not grant us a huge advantage in this case in either find_min or insert. This tells us that the unbalanced tree may actually be pretty well-balanced. Especially since the input is random. HOWEVER, if we sort the input before building when measuring the insert time, an AVL tree performs much much better than an unbalanced tree (linked list).

In [6]:
class DisjointSet:
# Credit: Copilot :(
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def make_set(self, key):
        self.parent[key] = key
        self.rank[key] = 0

    def find(self, key):
        if self.parent[key] != key: # Path compression
            self.parent[key] = self.find(self.parent[key])
        return self.parent[key]

    def union(self, key1, key2):
        root1 = self.find(key1)
        root2 = self.find(key2)

        if root1 != root2:
            if self.rank[root1] > self.rank[root2]:
                self.parent[root2] = root1
            elif self.rank[root1] < self.rank[root2]:
                self.parent[root1] = root2
            else:
                self.parent[root1] = root2
                self.rank[root2] += 1

Now testing the disjoint set:

In [7]:
ds = DisjointSet()
for i in range(1, 6):
    ds.make_set(i)

print(ds.find(1)) # Expected: 1
ds.union(1, 2)
print(ds.find(2)) # Expected: 2
ds.union(2, 3)
print(ds.find(3)) # Expected: 2

1
2
2


- ds.make_set is O(1) because insertion to dictionary is O(1).
- As we learned in the lectures, due to path compression, the first couple times running ds.find may take some time, but in the end it is just a dictionary look-up, so its ~O(1). 
- ds.union is ~O(1) because ds.find is ~O(1). The rest of this method is just shifting around the roots.

*Dynamic Programming* (DP) is a technique of solving problems by breaking it down into several smaller problems that overlap, and thus by caching the solution to one sub-problem, we can skip the computation of many sub-problems.
Example problems that can be solved with DP: Fibonacci numbers, Rod cutting, Matrix Chain Multiplication, Knapsack, Activity Selection

For DP to apply, a problem needs to fulfill two conditions:
- An optimal solution can be built from optimal solutions of sub-problems.
- Sub-problems can occur multiple times in the breakdown structure of the problem. (Overlapping sub-problems)

Solution strategies for DP-able problems include:
- Memoization (Top-down): start from the final problem, then break down step by step and record the solutions to each sub-problem until you reach a base case.
- Tabulation (Bottom-up): build the solution from the bottom, starting with the base cases and keep going until you reach the required state. Think of a for-loop. Can potentially save time in array look-ups and memory since we maaaybe don't need long-running dependencies. Depends on problem.
- 

In [ ]:
def fib_naive(n):
    # This also serves as a recursive relation formulation.
    if n <= 1:
        return n
    return fib_naive(n-1) + fib_naive(n-2)

fib_naive(10)

55

In [9]:
# Top down memoization
def fib_memoization(n, memo={}):
    if n in memo:
        return memo[n]
    if n <= 1:
        return n
    memo[n] = fib_memoization(n-1, memo) + fib_memoization(n-2, memo)
    return memo[n]

def fib_memoization2(n, memo={}):
    if n in memo:
        return memo[n]
    if n <= 1:
        return n
    memo[n] = fib_memoization2(n-2, memo) + fib_memoization2(n-1, memo)
    return memo[n]

print(fib_memoization(10))
print(fib_memoization2(10))

55
55


In [10]:
# Bottom up dynamic programming
# Typically this requires more insight into the problem structure
def fib_bottom_up(n):
    if n <= 1:
        return n
    fib = [0] * (n + 1)
    fib[1] = 1
    for i in range(2, n + 1):
        fib[i] = fib[i - 1] + fib[i - 2]
    return fib[n]

def fib_bottom_up_optimized(n):
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

print(fib_bottom_up(10))
print(fib_bottom_up_optimized(10))

55
55


In [18]:
runtime(
    fib_naive,
    name="Naive Fibonacci",
    iterations=5,
    n=30
    )

runtime(
    fib_memoization,
    name="Memoized Fibonacci",
    iterations=5,
    n=30
    )

runtime(
    fib_bottom_up,
    name="Bottom-up Fibonacci",
    iterations=5,
    n=30
    )

_=runtime(
    fib_bottom_up_optimized,
    name="Optimized Bottom-up Fibonacci",
    iterations=5,
    n=30
    )

Runtime of Naive Fibonacci: 2.629696846008301 seconds
Runtime of Memoized Fibonacci: 0.0 seconds
Runtime of Bottom-up Fibonacci: 0.0 seconds
Runtime of Optimized Bottom-up Fibonacci: 0.0 seconds


In [36]:
runtime(
    fib_memoization,
    name="Memoized Fibonacci (large n)",
    iterations=50,
    n=400
    )

runtime(
    fib_bottom_up,
    name="Bottom-up Fibonacci (large n)",
    iterations=50,
    n=400
    )

_=runtime(
    fib_bottom_up_optimized,
    name="Optimized Bottom-up Fibonacci (large n)",
    iterations=50,
    n=400
    )

Runtime of Memoized Fibonacci (large n): 0.0 seconds
Runtime of Bottom-up Fibonacci (large n): 0.006003141403198242 seconds
Runtime of Optimized Bottom-up Fibonacci (large n): 0.0020084381103515625 seconds


We see the list operations overhead is dragging down the un-optimized Bottom-Up Fibonacci, making it 3x as slow as the Optimized Bottom-up Fibonacci.

In the case of Fibonacci numbers, we cannot use a greedy approach, since there is not a branching path where we can make choices.

In [38]:
def cut_rod_naive(p, n):
    if n == 0:
        return 0
    q = float('-inf')
    for i in range(1, n + 1):
        if i <= len(p):
            q = max(q, p[i - 1] + cut_rod_naive(p, n - i))
    return q

def cut_rod_memoization(p, n, memo={}):
    if n in memo:
        return memo[n]
    if n == 0:
        return 0
    q = float('-inf')
    for i in range(1, n + 1):
        if i <= len(p):
            q = max(q, p[i - 1] + cut_rod_memoization(p, n - i, memo))
    memo[n] = q
    return q

def cut_rod_bottom_up(p, n):
    r = [0] * (n + 1)
    for j in range(1, n + 1):
        q = float('-inf')
        for i in range(1, j + 1):
            if i <= len(p):
                q = max(q, p[i - 1] + r[j - i])
        r[j] = q
    return r[n]

p = [1, 5, 8, 9, 10, 17, 17, 20]
n = 8
print(cut_rod_naive(p, n))
print(cut_rod_memoization(p, n))
print(cut_rod_bottom_up(p, n))

runtime(
    cut_rod_naive,
    name="Naive Cut Rod",
    iterations=5,
    p=p,
    n=n
    )

_=runtime(
    cut_rod_memoization,
    name="Memoized Cut Rod",
    iterations=50,
    p=p,
    n=n
    )

_=runtime(
    cut_rod_bottom_up,
    name="Bottom-up Cut Rod",
    iterations=50,
    p=p,
    n=n
    )

22
22
22
Runtime of Naive Cut Rod: 0.001999378204345703 seconds
Runtime of Memoized Cut Rod: 0.0 seconds
Runtime of Bottom-up Cut Rod: 0.0020020008087158203 seconds


Rod-cutting cannot be solved using a greedy approach, since there is no locally optimal solution to a sub-problem that can lead to an optimal solution of the whole. In particular, the optimal solution of a sub-problem would lead the greedy algorithm to cut with the highest available reward per unit, however, the graph of marginal reward may not be monotonically increasing (or decreasing for that matter), which would therefore trick the algorithm to make sub-optimal choices.

In [51]:
def greedy_cut_rod(p, n):
    cuts = []
    price_total = 0
    while n > 0:
        max_price = float('-inf')
        cut_length = 0
        for i in range(1, min(len(p), n) + 1):
            if p[i - 1] > max_price:
                max_price = p[i - 1]
                cut_length = i
        cuts.append(cut_length)
        price_total += max_price
        n -= cut_length
    return price_total

reward_per_unit = [1, 2, 3, 4, 10, 11, 12, 13] # Monotonic
p = [r*i for i, r in enumerate(reward_per_unit, start=1)]
print(greedy_cut_rod(p, n))
print(cut_rod_naive(p, n))
# It works!

reward_per_unit = [1, 2, 3, 4, 10, 9, 8, 7] # Not monotonic
p = [r*i for i, r in enumerate(reward_per_unit, start=1)]
print(greedy_cut_rod(p, n))
print(cut_rod_naive(p, n))
# It fails!

104
104
57
59


Dynamic programming solves problems by breaking them into overlapping subproblems, solving each once, and combining their results to guarantee a globally optimal solution. It trades extra memory and computation for correctness and is well suited to problems where early decisions affect later outcomes.

Greedy algorithms make the best local choice at each step, aiming for speed and simplicity. They are efficient but only correct when the problem has the greedy-choice property. Otherwise, locally optimal decisions may lead to suboptimal overall results, as we saw in the case above with rod-cutting.